In [1]:
import pandas as pd
import numpy as np
from scipy.stats import poisson

In [11]:
def main():
    #Intro. Datasets containing 1)stats 2)Bookmakers odds 3)sheet to match team names between 1&2 need to be read in
    gbpstake = 40
    df_understat = pd.read_excel(r'C:\Users\jakub\Documents\kursy\python\python_understat\understat_big5_2023.xlsx')
    df_bf_match = pd.read_excel(r'C:\Users\jakub\Documents\kursy\python\python_understat\betfair_prices_big5_match.xlsx', index_col = 0, sheet_name=['MatchOdds', 'DnB','DCh'])
    bf_match_odds = df_bf_match['MatchOdds']
    bf_dnb_odds = df_bf_match['DnB']
    bf_dch_odds = df_bf_match['DCh']
    
    df_bf_undersovers = pd.read_excel(r'C:\Users\jakub\Documents\kursy\python\python_understat\betfair_prices_big5.xlsx', index_col = 0)
    
    for df in [bf_dnb_odds,bf_dch_odds,bf_match_odds,df_bf_undersovers]:
        df = df.drop(columns=['BetID'],inplace=True)
   
    bf_names = pd.read_excel(r'C:\Users\jakub\Documents\kursy\python\python_understat\bf_understat_match_names.xlsx')
    df_understat_x_bf_names = pd.merge(df_understat, bf_names, on='team_name', how='outer')
    #Only keeps statistics needed
    df_xg_stats_h = df_understat_x_bf_names[['betfair_names','h_npxg','npxg']]
    df_xg_stats_a = df_understat_x_bf_names[['betfair_names','a_npxg','npxg']]
    df_xg_stats_h = df_xg_stats_h.rename(columns={"betfair_names": "Home","npxg": "npxg_1"})
    df_xg_stats_a = df_xg_stats_a.rename(columns={"betfair_names": "Away","npxg": "npxg_2"})
   
    bf_match_odds_x_df_xg_stats_ha = merge_dfs_inner(bf_match_odds,df_xg_stats_h,df_xg_stats_a,'Home','Away')
    bf_dnb_odds_x_df_xg_stats_ha = merge_dfs_inner(bf_dnb_odds,df_xg_stats_h,df_xg_stats_a,'Home','Away')
    df_bf_undersovers_x_df_xg_stats_ha = merge_dfs_inner(df_bf_undersovers,df_xg_stats_h,df_xg_stats_a,'Home','Away')
    
    #Part 1: Match Outcome Odds ("MatchOdds"): home win, away win or draw
    #Step 1 calculates probabilities
    temp = probs_table("MatchOdds",bf_match_odds_x_df_xg_stats_ha)
    [Home,p_home_win,p_away_win,p_draw] = [temp[0],temp[1],temp[2],temp[3]]
    
    col_name = ['Home','p_home_win','p_away_win','p_draw']
    df_match_prob = pd.DataFrame([Home.tolist(),p_home_win,p_away_win,p_draw],index=col_name)
    df_match_prob = df_match_prob.T
    bf_match_odds_probs = pd.merge(bf_match_odds_x_df_xg_stats_ha, df_match_prob, on='Home', how='inner')
    #Step 2 calculates Expected Value of a bet. Calculates Kelly criterion of an optimal bet stake
    temp = EV_kelly(bf_match_odds_probs.home_win,bf_match_odds_probs.p_home_win)
    [EV_home_win,KR_home_win] = [temp[0],temp[1]]
    temp = EV_kelly(bf_match_odds_probs.away_win,bf_match_odds_probs.p_away_win)
    [EV_away_win,KR_away_win] = [temp[0],temp[1]]
    temp = EV_kelly(bf_match_odds_probs.draw,bf_match_odds_probs.p_draw)
    [EV_draw,KR_draw] = [temp[0],temp[1]]
    bf_match_odds_probs = bf_match_odds_probs.assign(EV_home_win = EV_home_win, EV_away_win = EV_away_win, EV_draw = EV_draw,KR_home_win = KR_home_win, KR_away_win = KR_away_win, KR_draw = KR_draw)
    #Step 3 calculates actually bet stake.   
    [stake_home_win,stake_away_win,stake_draw] = [bet_stake(bf_match_odds_probs.KR_home_win,gbpstake),bet_stake(bf_match_odds_probs.KR_away_win,gbpstake),bet_stake(bf_match_odds_probs.KR_draw,gbpstake)]
    bf_match_odds_probs = bf_match_odds_probs.assign(stake_home_win = stake_home_win, stake_away_win = stake_away_win, stake_draw = stake_draw)
    #Part 2: Match Outcome Odds DrawNoBet: home_dnb_win or away_dnb_win. Steps as in part1
    temp = probs_table("DnB",bf_dnb_odds_x_df_xg_stats_ha)
    [Home,p_home_win,p_away_win,p_draw,p_home_win_dnb,p_away_win_dnb] = [temp[0],temp[1],temp[2],temp[3],temp[4],temp[5]]
    
    col_name = ['Home','p_home_win','p_away_win','p_home_win_dnb','p_away_win_dnb']
    df_match_dnb_prob = pd.DataFrame([Home.tolist(),p_home_win,p_away_win,p_home_win_dnb,p_away_win_dnb],index=col_name)
    df_match_dnb_prob = df_match_dnb_prob.T

    bf_dnb_odds_probs = pd.merge(bf_dnb_odds_x_df_xg_stats_ha, df_match_dnb_prob, on='Home', how='inner')
    bf_dnb_odds_probs = bf_dnb_odds_probs.rename(columns={"Over_price": "home_dnb", "Under_price": "away_dnb"})
    
    temp = EV_kelly(bf_dnb_odds_probs.Home_dnb,bf_dnb_odds_probs.p_home_win_dnb)
    [EV_home_win_dnb,KR_home_win_dnb] = [temp[0],temp[1]]
    temp = EV_kelly(bf_dnb_odds_probs.Away_dnb,bf_dnb_odds_probs.p_away_win_dnb)
    [EV_away_win_dnb,KR_away_win_dnb] = [temp[0],temp[1]]
    #Part 3: Overs/ Unders - Number of goals scored in the game.Steps as in part1
    bf_dnb_odds_probs = bf_dnb_odds_probs.assign(EV_home_win_dnb = EV_home_win_dnb, EV_away_win_dnb = EV_away_win_dnb,KR_home_win_dnb = KR_home_win_dnb, KR_away_win_dnb = KR_away_win_dnb)    
    
    [stake_home_win_dnb,stake_away_win_dnb] = [bet_stake(bf_dnb_odds_probs.KR_home_win_dnb,gbpstake),bet_stake(bf_dnb_odds_probs.KR_away_win_dnb,gbpstake)]
    bf_dnb_odds_probs = bf_dnb_odds_probs.assign(stake_home_win_dnb = stake_home_win_dnb, stake_away_win_dnb = stake_away_win_dnb)

    temp = probs_table("Overs",df_bf_undersovers_x_df_xg_stats_ha)
    [Home,BetName,p_under,p_over] = [temp[0],temp[1],temp[2],temp[3]]
    
    col_name = ['Home','BetName','p_under','p_over']
    df_overs_prob = pd.DataFrame([Home.tolist(),BetName.tolist(),p_under,p_over],index=col_name)
    df_overs_prob = df_overs_prob.T
    bf_overs_odds_probs = pd.merge(df_bf_undersovers_x_df_xg_stats_ha, df_overs_prob, on=['Home','BetName'], how='inner')
    
    temp = EV_kelly(bf_overs_odds_probs.Under_price,bf_overs_odds_probs.p_under)
    [EV_under_win,KR_under] = [temp[0],temp[1]]
    temp = EV_kelly(bf_overs_odds_probs.Over_price,bf_overs_odds_probs.p_over)
    [EV_over_win,KR_over] = [temp[0],temp[1]]
    bf_overs_odds_probs = bf_overs_odds_probs.assign(EV_under = EV_under_win, EV_over = EV_over_win,KR_under = KR_under, KR_over = KR_over)
    
    [stake_under,stake_over] = [bet_stake(bf_overs_odds_probs.KR_under,gbpstake),bet_stake(bf_overs_odds_probs.KR_over,gbpstake)]
    bf_overs_odds_probs = bf_overs_odds_probs.assign(stake_under = stake_under, stake_over = stake_over)
    
    for df in [bf_overs_odds_probs,bf_match_odds_probs,bf_dnb_odds_probs]:
        for column in df.columns[6:len(df.columns)]:
            df[column] = df[column].apply(lambda x: round(x, 2))
            
    path1 =  r'C:/Users/jakub/Documents/kursy/python/python_understat/poisson_matchodds.xlsx'    
    path2 =  r'C:/Users/jakub/Documents/kursy/python/python_understat/poisson_overs.xlsx' 

    with pd.ExcelWriter(path1) as writer:
        bf_match_odds_probs.to_excel(writer, sheet_name="MatchOdds")
        bf_dnb_odds_probs.to_excel(writer, sheet_name="DnbOdds")
    with pd.ExcelWriter(path2) as writer:
        bf_overs_odds_probs.to_excel(writer, sheet_name="OverOdds")

In [3]:
def merge_dfs_inner(df_a,df_b,df_c,joint1,joint2):
    temp = pd.merge(df_a,df_b,on = joint1, how = 'inner')
    df_mrg = pd.merge(temp,df_c, on = joint2, how = 'inner')
    return df_mrg

In [4]:
def probs_table(market,mat):
    match market:
        case "MatchOdds":
            p_home_win = []
            p_away_win = []
            p_draw = []
            Home = mat.Home
            p_home_win_dnb = []
            p_away_win_dnb = []

            for count in range(0,len(mat.index)):
                temp = score(count,mat)
                p_home_win.append(temp[0])
                p_away_win.append(temp[1])
                p_draw.append(temp[2])
            return Home, p_home_win, p_away_win, p_draw
        
        case "DnB":
            p_home_win = []
            p_away_win = []
            p_draw = []
            Home = mat.Home
            p_home_win_dnb = []
            p_away_win_dnb = []
    
            for count in range(0,len(mat.index)):
                temp = score(count,mat)
                p_home_win.append(temp[0])
                p_away_win.append(temp[1])
                p_draw.append(temp[2])
                p_home_win_dnb.append(temp[0]/(temp[0]+temp[1]))
                p_away_win_dnb.append(temp[1]/(temp[0]+temp[1]))
            return Home, p_home_win, p_away_win, p_draw, p_home_win_dnb, p_away_win_dnb
        
        case "Overs":
            counter = 0
            p_under = []
            p_over = []
            markets = []  
            Home = mat.Home
            BetName = mat.BetName
    
            for count in range(0,len(mat.index)):
                temp = score_underover(mat.BetName[count],count,mat)
                count +=6
                p_under.append(temp[0])
                p_over.append(temp[1])
            return Home, BetName, p_under, p_over

In [5]:
def score(team_no,odds_stats_matrix):
    goals_home = []
    goals_away = []
    goals_home_general = []
    goals_away_general = []   

    for count in range (0,10):
        goals_home.append(poisson.pmf(k=count, mu=odds_stats_matrix['h_npxg'].iloc[team_no]))
        goals_away.append(poisson.pmf(k=count, mu=odds_stats_matrix['a_npxg'].iloc[team_no]))
        goals_home_general.append(poisson.pmf(k=count, mu=odds_stats_matrix['npxg_1'].iloc[team_no]))
        goals_away_general.append(poisson.pmf(k=count, mu=odds_stats_matrix['npxg_2'].iloc[team_no]))
    
    goals_home_matrix = np.matrix(goals_home)
    goals_away_matrix = np.matrix(goals_away)
    
    goals_home_matrix = np.transpose(goals_home_matrix)
    
    score_matrix = np.matmul(goals_home_matrix,goals_away_matrix)
    
    return winner_prob(score_matrix)

In [6]:
def winner_prob(input):
    p_draw = np.sum(np.diagonal(input))
    p_home_win = np.sum(np.tril(input)) - p_draw
    p_away_win = np.sum(np.triu(input)) - p_draw
    return p_home_win, p_away_win, p_draw

In [7]:
def score_underover(market,team_no,odds_stats_matrix):
    goals_home = []
    goals_away = []
    goals_home_general = []
    goals_away_general = []   

    for count in range (0,10):
        goals_home.append(poisson.pmf(k=count, mu=odds_stats_matrix['h_npxg'].iloc[team_no]))
        goals_away.append(poisson.pmf(k=count, mu=odds_stats_matrix['a_npxg'].iloc[team_no]))
        goals_home_general.append(poisson.pmf(k=count, mu=odds_stats_matrix['npxg_1'].iloc[team_no]))
        goals_away_general.append(poisson.pmf(k=count, mu=odds_stats_matrix['npxg_2'].iloc[team_no]))
              
    goals_home_matrix = np.matrix(goals_home)
    goals_away_matrix = np.matrix(goals_away)
    
    goals_home_matrix = np.transpose(goals_home_matrix)
    
    score_matrix = np.matmul(goals_home_matrix,goals_away_matrix)

    p_under = goals_scored(market,score_matrix)[0]
    p_over = goals_scored(market,score_matrix)[1]

    return p_under, p_over

In [8]:
def goals_scored(market,matr):
    match market:
        case "Over/Under 0.5 Goals":
            p_under = matr[0,0]
        case "Over/Under 1.5 Goals":
            p_under = matr[0,0] + matr[1,0] + matr[0,1]                  
        case "Over/Under 2.5 Goals":
            p_under = matr[0,0] + matr[1,0] + matr[0,1] + matr[1,1] + matr[2,0] + matr[0,2]                     
        case "Over/Under 3.5 Goals":
            p_under = matr[0,0] + matr[1,0] + matr[0,1] + matr[1,1] + matr[2,0] + matr[0,2] + + matr[2,1] + +matr[1,2] + matr[3,0] + matr[0,3]
        case "Over/Under 4.5 Goals":
            p_under = matr[0,0] + matr[1,0] + matr[0,1] + matr[1,1] + matr[2,0] + matr[0,2] + + matr[2,1] + +matr[1,2] + matr[3,0] + matr[0,3] + matr[4,0] + matr[0,4] + matr[2,2] + matr[3,1] + matr[1,3]                  
        case "Both teams to Score?":
            p_under = np.sum(matr[0,:]) + np.sum(matr[:,0]) - matr[0,0]
    p_over = 1- p_under
    return p_under, p_over

In [9]:
def EV_kelly(odds,prob):
    EV = (odds-1)*prob - (1-prob)
    KR = (odds * prob -1) / (odds - 1)
    return EV,KR

In [10]:
def bet_stake(kellyratio,factor):
    stake = []
    for elem in kellyratio:
        if elem < 0:
            elem = 0
        stake.append(elem*factor)
    return stake

In [12]:
main()